   > ## Importation des librairies:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

> ## Importation des algorithmes:

In [2]:
from sklearn import datasets, linear_model #sklearn est une mine d'or pour faire du machine learning dans Python.
from sklearn.metrics import mean_squared_error

> ## Importation du jeu de donnees et description:

In [3]:
from sklearn.datasets import load_diabetes
#On doit definir quel jeu de donnees precisement.

In [4]:
diabetes=load_diabetes()
# On installe le jeu de donnees

In [5]:
diabetes.keys()# les attributs du jeu de donnees

dict_keys(['data', 'target', 'DESCR', 'feature_names'])

In [47]:
print (diabetes.DESCR) # Description du jeu de donnees.

Diabetes dataset

Notes
-----

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

Data Set Characteristics:

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attributes:
    :Age:
    :Sex:
    :Body mass index:
    :Average blood pressure:
    :S1:
    :S2:
    :S3:
    :S4:
    :S5:
    :S6:

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bradley Efron, Trevor Hastie, Iain Johnstone and Robert Tibshirani

In [6]:
#A quoi cela ressemble?
#Notez que Python commence a 0, pas 1. Donc quand vous voulez sortir les 3 premieres rangees, commencez a 0.
print (diabetes.feature_names)
print (diabetes.data[0:3])
print (diabetes.target[0:3])


['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
[[ 0.03807591  0.05068012  0.06169621  0.02187235 -0.0442235  -0.03482076
  -0.04340085 -0.00259226  0.01990842 -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 -0.02632783 -0.00844872 -0.01916334
   0.07441156 -0.03949338 -0.06832974 -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 -0.00567061 -0.04559945 -0.03419447
  -0.03235593 -0.00259226  0.00286377 -0.02593034]]
[151.  75. 141.]


> ## Definition de la variable a predire:

In [7]:
df_diabetes = pd.DataFrame(diabetes.data,columns=diabetes.feature_names)#Transposons nos 10 features dans une dataframe: colonnes et rangees
df_diabetes['target'] = pd.Series(diabetes.target)# Transposons notre variable a predire dans une serie
y = diabetes.target #Definissons y comme etant notre variable a predire. y ici est une donnee quantitative mesurant le degre de progression de la maladie.
df_diabetes.head(5)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


> ## Exploration des donnees ou EDA: Exploratory Data Analysis:

In [8]:
#Definition de x et y
X=diabetes.data
y=diabetes.target

Standardisons nos donnees
La plupart des algorithmes de machine learning fonctionnent "mieux" lorsque les donnees sont standardisees ou normalisees. Qu'est ce que la standardisation/normalisation des donnees? Et pourquoi pensez-vous que c'est important?

Reponses:
Formules mathematiques:
1) Standardisation: (x- moyenne x)/ecart-type

2) Normalisation: (x-valeur min x)/(valeur max x -valeur min x)

Beaucoup d'algorithmes utilisent le concept de distance et donc si certaines variables x ont de tres grandes valeurs eloignees les unes des autres, cela peut fausser le processus d'apprentissage.

L'algorithme KNN compare la distance (mathematique) entre les coordonnees x de l'observation dont on veut predire la classe (dans le jeu de donnees test) par rapport aux k autres observations dans le jeu de donnees d'apprentissage. La classe attribuee a l'observation en question est la classe majoritaire des k observations. Quand il y a plusieurs x, (variables independentes) et que celles-ci sont ont des valeurs tres eloignees les unes des autres par exemple la masse et les degres pour predire l'appartenance a telle ou telle categories d'etoiles...Les valeurs masse varient entre 10^29 et 10^32, les valeurs temperatures varient entre 6000 degres et 100000 degres. Dans le calcul de la distance les valeurs masse vont tres largement dominer ce qui fait les valeurs degres ne vont presque pas compter.

Tout ca pour expliquer le fonctionnement de KNN (simple) et pour demontrer l'importance des "distances".

Sachez qu'il existe plusieurs distances mathematiques: Euclidean, Manhattan, Minkowski. NB: Euclidean (la plus utilisee est Minkowski ou p=2 et Manhattan quand p=1). KNN a tendance a utiliser la distance euclidienne.

Importons donc une classe destinee a standardiser nos donnees, nous allons ici utiliser la standardisation

In [10]:
from sklearn.preprocessing import StandardScaler  

In [11]:
scaler = StandardScaler()
Xss = scaler.fit_transform(X)
Xss = pd.DataFrame(data=Xss,columns= diabetes['feature_names'])
Xss.describe()
Xss.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.800500,1.065488,1.297088,0.459840,-0.929746,-0.732065,-0.912451,-0.054499,0.418551,-0.370989
1,-0.039567,-0.938537,-1.082180,-0.553511,-0.177624,-0.402886,1.564414,-0.830301,-1.436551,-1.938479
2,1.793307,1.065488,0.934533,-0.119218,-0.958674,-0.718897,-0.680245,-0.054499,0.060207,-0.545154
3,-1.872441,-0.938537,-0.243771,-0.770658,0.256292,0.525397,-0.757647,0.721302,0.477072,-0.196823
4,0.113172,-0.938537,-0.764944,0.459840,0.082726,0.327890,0.171178,-0.054499,-0.672582,-0.980568


In [12]:
from sklearn.preprocessing import normalize

Xn = normalize(X)

In [14]:
Xn = pd.DataFrame(data=Xn,columns= diabetes['feature_names'])
Xn.describe()
Xn.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.321006,0.427268,0.520141,0.184399,-0.372834,-0.293563,-0.365899,-0.021855,0.167842,-0.148769
1,-0.011662,-0.276616,-0.318952,-0.163137,-0.052351,-0.118743,0.461081,-0.244715,-0.423396,-0.571330
2,0.657407,0.390596,0.342590,-0.043704,-0.351439,-0.263540,-0.249370,-0.019979,0.022071,-0.199848
3,-0.721099,-0.361441,-0.093879,-0.296789,0.098701,0.202336,-0.291778,0.277782,0.183726,-0.075799
4,0.062769,-0.520546,-0.424265,0.255044,0.045883,0.181859,0.094941,-0.030227,-0.373038,-0.543858


> ## Demarrage de la phase de test: 

In [15]:
from sklearn.model_selection import train_test_split 

**C'est la ou la partie learning de Machine Learning rentre en scene!**

Remarquez aussi **split**, cela signifie que l'on va diviser notre jeu de donnee en deux: une partie pour la premiere phase qui correspond a la phase d'**l'entrainement** de la machine et l'autre partie pour **tester** si ce que la machine aura appris est fiable.

In [72]:
Xss_train, Xss_test, y_train, y_test = train_test_split(Xss, y, test_size=0.2, random_state=42) 

In [73]:
print (Xss_train.shape)
print (Xss_test.shape)
print (y_train.shape)
print (y_test.shape)



(353, 10)
(89, 10)
(353,)
(89,)


## Installation du modele de regression lineaire sur le jeu de donnees: ##

In [74]:
# On cree une instance de notre modele
lm1 = linear_model.LinearRegression()

Xss = diabetes.data
y = diabetes.target 
#On fit/installe notre modele sur notre jeu de donnees.
lm1.fit(Xss_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [75]:
print (lm1.coef_)#Interpreter la signification des coefficients
# + 1 an, se traduit par une augmentation de 37 de l'indice diabetique/la progression de la maladie apres 1 an.
# une augmentation d'un point d'indice de masse corporelle se traduit par une augmentation de 542 de l'indice diabetique.

[  37.90031426 -241.96624835  542.42575342  347.70830529 -931.46126093
  518.04405547  163.40353476  275.31003837  736.18909839   48.67112488]


In [76]:
print (lm1.intercept_) # sans aucune influence des facteurs sexe, age etc...l'indice diabetique est de 151.

151.3456553477407


In [77]:
pred1 = lm1.predict(Xss_test)
#Ici, on sort les 10 premieres valeurs predites par notre learning post apprentissage...
print(lm1.predict(X_test[0:10]))

[139.5483133  179.52030578 134.04133298 291.41193598 123.78723656
  92.17357677 258.23409704 181.33895238  90.22217862 108.63143298]


> ### Evaluation du modele:

> #### 1) Calcul de l'erreur du modele:

In [78]:
print (np.sqrt(mean_squared_error(y_test, pred1)))

53.8532569849144


> #### 2) Calcul du score du modele:

In [79]:
score1=lm1.score(Xss_test,y_test)
print (score1)


0.452606602161738


> #### 3) Calcul de l'erreur du modele de base:

In [27]:
#baseline prediction
#Ici, le modele de base/notre benchmark est le modele qui va predire que toute nouvelle observation la moyenne 
baseline=np.mean(y_test)
baseline



145.77528089887642

In [28]:
baseline_pred=np.empty(len(y_test))# cree un nouveau tableau
baseline_pred.fill(baseline) #rempli avec predictions
print(np.sqrt(mean_squared_error(y_test,baseline_pred)))

72.78840394263774


In [29]:
print(y_test, baseline_pred)
#Ici, on sort les valeurs predies versus la moyenne

[219.  70. 202. 230. 111.  84. 242. 272.  94.  96.  94. 252.  99. 297.
 135.  67. 295. 264. 170. 275. 310.  64. 128. 232. 129. 118. 263.  77.
  48. 107. 140. 113.  90. 164. 180. 233.  42.  84. 172.  63.  48. 108.
 156. 168.  90.  52. 200.  87.  90. 258. 136. 158.  69.  72. 171.  95.
  72. 151. 168.  60. 122.  52. 187. 102. 214. 248. 181. 110. 140. 202.
 101. 222. 281.  61.  89.  91. 186. 220. 237. 233.  68. 190.  96.  72.
 153.  98.  37.  63. 184.] [145.7752809 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809
 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809
 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809
 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809
 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809
 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809
 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809 145.7752809
 145.7752809 145.7752809 145.7752809

### > Interpretation ###